# 合并数据集：合并与连接
## 关系代数
pd.merge()实现的功能基于关系代数（relational algebra）的一部分。

## 数据连接的类型
pd.merge()函数实现了三种数据连接的类型：一对一、多对一和多对多。

### 一对一连接

In [64]:
# 一对一连接
import pandas as pd

df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'], 
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'], 
                    'hire_date': [2004, 2008, 2012, 2014]})
print(df1);print(df2)

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014


In [65]:
df3 = pd.merge(df1,  df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


### 多对一连接

In [66]:
# 多对一连接
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'], 
                    'supervisor': ['Carly', 'Guido', 'Steve']})
print(df3);print(df4);print(pd.merge(df3, df4))

  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve


### 多对多连接
多对多的概念有点复杂，如果左右两个输入的共同列都包含重复值，那么合并的结果就是一种多对多连接。

# 通过多对多链接，就可以得知每位员工所具备的能力：

In [67]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting', 
                              'Engineering', 'Engineering', 
                              'HR', 'HR'], 
                    'skills': ['math', 'spreadsheets', 
                               'coding', 'linux', 
                               'spreadsheets', 'organization']})

print(df1);print(df5);
print(pd.merge(df1, df5))

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
         group        skills
0   Accounting          math
1   Accounting  spreadsheets
2  Engineering        coding
3  Engineering         linux
4           HR  spreadsheets
5           HR  organization
  employee        group        skills
0      Bob   Accounting          math
1      Bob   Accounting  spreadsheets
2     Jake  Engineering        coding
3     Jake  Engineering         linux
4     Lisa  Engineering        coding
5     Lisa  Engineering         linux
6      Sue           HR  spreadsheets
7      Sue           HR  organization


## 设置数据合并的键
### 参数on的用法
最简单的方法就是直接将参数on设置为一个列名字符串或者一个包含多列名称的列表：

In [68]:
print(df1);print(df2);print(pd.merge(df1, df2, on='employee'))

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014


### left_on与right_on参数

In [69]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'], 
                    'salary': [70000, 80000, 120000, 9000]})
print(df1); print(df3);
print(pd.merge(df1, df3, left_on = "employee", right_on = "name"))

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue    9000
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue    9000


In [70]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,9000


### left_index与right_index参数

In [71]:
# 合并索引
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
print(df1a);print(df2a)

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR
          hire_date
employee           
Lisa           2004
Bob            2008
Jake           2012
Sue            2014


还可以通过设置pd.merge()中的left_index和/或right_index参数将索引设置为键来实现合并:

In [73]:
print(df1a);print(df2a);
print(pd.merge(df1a, df2a, left_index = True, right_index=True))

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR
          hire_date
employee           
Lisa           2004
Bob            2008
Jake           2012
Sue            2014
                group  hire_date
employee                        
Bob        Accounting       2008
Jake      Engineering       2012
Lisa      Engineering       2004
Sue                HR       2014


为了方便考虑，DataFrame实现了join()方法，它可以按照索引进行数据合并

In [74]:
print(df1a);print(df2a);print(df1a.join(df2a))

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR
          hire_date
employee           
Lisa           2004
Bob            2008
Jake           2012
Sue            2014
                group  hire_date
employee                        
Bob        Accounting       2008
Jake      Engineering       2012
Lisa      Engineering       2004
Sue                HR       2014


如果想将索引与列混合起来用，可以通过结合left_index与right_on, 或者结合left_on与right_index来实现：

In [75]:
print(df1a);print(df3);
print(pd.merge(df1a, df3, left_index=True, right_on='name'))

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue    9000
         group  name  salary
0   Accounting   Bob   70000
1  Engineering  Jake   80000
2  Engineering  Lisa  120000
3           HR   Sue    9000


## 设置数据连接的集合操作

In [76]:
df6 = pd.DataFrame({'name':['Peter', 'Paul', 'Mary'], 
              'food':['fish', 'beans', 'bread']}, 
             columns = ['name', 'food'])

df7 = pd.DataFrame({'name': ['Mary', 'Joseph'], 
                    'drink': ['wine', 'beer']}, 
                   columns = ['name', 'drink'])

print(df6);print(df7);print(pd.merge(df6, df7))

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread
     name drink
0    Mary  wine
1  Joseph  beer
   name   food drink
0  Mary  bread  wine


In [77]:
pd.merge(df6, df7, how = 'inner')

,name,food,drink
0,Mary,bread,wine


In [78]:
print(df6);print(df7);print(pd.merge(df6, df7, how = 'outer'))

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread
     name drink
0    Mary  wine
1  Joseph  beer
     name   food drink
0   Peter   fish   NaN
1    Paul  beans   NaN
2    Mary  bread  wine
3  Joseph    NaN  beer


In [79]:
print(df6);print(df7);print(pd.merge(df6, df7, how = 'left'))

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread
     name drink
0    Mary  wine
1  Joseph  beer
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine


## 重复列名：suffixes参数
你可能会遇到两个输入DataFrame有重命名列的情况。代码如下：


In [80]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'], 
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'], 
                    'rank': [3, 1, 4, 2]})
print(df8);print(df9);
print(pd.merge(df8, df9, on="name"))

   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2
   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2


In [81]:
print(df8);print(df9);
print(pd.merge(df8, df9, on="name", suffixes=["_L", "_R"]))

   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2
   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2


## 案例：美国各州的统计数据

使用下面的shell下载数据

- !curl -0 https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv

- !curl -0 https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv

- !curl -0 https://raw.gighubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv

In [82]:
# 用Pandas的read_csv分别读取这三个数据集
pop = pd.read_csv(r"D:\Python-Excel\Data-analysis\Python-Data-Science\Objects\data\state-population.csv")
areas = pd.read_csv(r"D:\Python-Excel\Data-analysis\Python-Data-Science\Objects\data\state-areas.csv")
abbrevs = pd.read_csv(r"D:\Python-Excel\Data-analysis\Python-Data-Science\Objects\data\state-abbrevs.csv")
print(pop.head()); print(areas.head());print(abbrevs.head())

  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA


首先用一个多对一合并获取人口(pop)DataFrame中各州名称缩写对应的全称。我们需要将pop的state/region列与abbrevs的abbreviation列进行合并，还需要
通过how='outer'确保数据没有丢失。

In [83]:
merged = pd.merge(pop, abbrevs, how='outer', 
                  left_on = 'state/region', 
                  right_on = 'abbreviation')

merged = merged.drop('abbreviation', 1)   # 丢弃重复信息
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


检查是否有缺失值

In [84]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

仔细检查是哪些数据缺失了

In [85]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


现在看看是哪些州有缺失

In [86]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

In [87]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [88]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [89]:
# 再检查一下数据，看看哪些列还有缺失值没有匹配上
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [90]:
# 面积area列里面还有缺失值，来看看究竟是哪些地区面积缺失
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

In [91]:
# 发现面积(areas)DataFrame里面不包含全美国的面积数据。可以插入全国总面积数据
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [92]:
# 我们先选择2010年的各州人口以及总人口数据，然后用query()函数进行快速计算
data2010 = final.query("year == 2010 &  ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


In [93]:
# 计算人口密度并按序排列。首先对索引进行重置，然后再计算结果：
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']

density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

In [94]:
# 查看人口密度最低的几个州的数据
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64